In [1]:
import json
import mailbox
import email

In [2]:
fn='messages/inbox/anna_d0ta1k5dmg/message_1.json'
jdata = json.load(open(fn))

In [3]:
def utfdemangle(s):
    # FB delivers strange JSON where all strings are encoded in UTF8 bytes
    return s.encode('latin1').decode('utf8')

In [4]:
def group_messages(jdata):
    # group messages accrodig to sender and time
    # so that messages sent shortly after each other 
    # end up as multiple paragraphs
    threshold_ms = 60000
    rawmsgs = jdata['messages']
    rawmsgs.sort(key=lambda m: m['timestamp_ms'])
    oldts = 0
    oldsender = None
    group = None
    groups = []
    for msg in rawmsgs:
        if msg['sender_name'] != oldsender or abs(msg['timestamp_ms'] - oldts) > threshold_ms:
            if group is not None:
                groups.append(group)
            group = { 'sender_name': msg['sender_name'], 'timestamp_ms': msg['timestamp_ms'], 'messages' : []}

        group['messages'].append(msg)
        oldts = msg['timestamp_ms']
        oldsender = msg['sender_name']

    groups.append(group)

    return groups

list??

In [6]:
m=email.message.EmailMessage()


In [7]:
m['From']=utfdemangle(msg['sender_name'])
m['To']='Group'
m.set_content(utfdemangle(msg['content']))
